# Inference Engine: vLLM

This notebook shows **how to run and measure interactive inference** with a local, fully-offline copy of *Mistral-Small-24B-Instruct* using the **vLLM** engine.  
We will:

1. set up a reproducible, internet-free environment;  
2. load the model in bfloat16 and define a concise *system* instruction that keeps replies short and direct;  
3. generate sample answers for two prompts to confirm everything is wired correctly;  
4. implement a lightweight benchmark that reports average latency and output-tokens-per-second;  

Feel free to tweak the prompts, sampling parameters, or `num_runs` variable to explore how temperature, max token count, and batch size affect throughput on your own hardware.


In [1]:
!nvidia-smi

Wed May  7 12:32:03 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.161.08             Driver Version: 535.161.08   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA H100 NVL                On  | 00000000:B1:00.0 Off |                    0 |
| N/A   32C    P0              58W / 400W |      0MiB / 95830MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [2]:
import os, time, torch
from vllm import LLM, SamplingParams               

os.environ["HF_HUB_OFFLINE"] = "1"                 # no outbound traffic
os.environ["TRANSFORMERS_OFFLINE"] = "1"
device = "cuda" if torch.cuda.is_available() else "cpu"


/software/python-anaconda-2022.05-el8-x86_64/envs/vllm_serving/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO 05-07 12:32:07 [__init__.py:239] Automatically detected platform cuda.


2025-05-07 12:32:08,912	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [3]:
model_path = "/project/rcc/shared/ai-workshops/Mistral-Small-24B-Instruct-2501"

system_prompt = (
    "You are a helpful assistant. You will be given a task and you should "
    "respond with a solution. You should be concise and clear. One plain "
    "paragraph—no lists, no headings, no filler."
)

prompts = [
    "Give me a short introduction to large language model inference.",
    "The benefits of artificial intelligence in healthcare include:"
]

In [4]:
sampling = SamplingParams(max_tokens=1024, temperature=0.7)   

llm = LLM(model=model_path)                                  


INFO 05-07 12:32:15 [config.py:717] This model supports multiple tasks: {'reward', 'generate', 'classify', 'embed', 'score'}. Defaulting to 'generate'.
INFO 05-07 12:32:15 [config.py:2003] Chunked prefill is enabled with max_num_batched_tokens=16384.
INFO 05-07 12:32:17 [core.py:58] Initializing a V1 LLM engine (v0.8.5) with config: model='/project/rcc/shared/ai-workshops/Mistral-Small-24B-Instruct-2501', speculative_config=None, tokenizer='/project/rcc/shared/ai-workshops/Mistral-Small-24B-Instruct-2501', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='auto', reasoning_backend=None), observability_c

Loading safetensors checkpoint shards:   0% Completed | 0/10 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  10% Completed | 1/10 [00:01<00:17,  1.90s/it]
Loading safetensors checkpoint shards:  20% Completed | 2/10 [00:03<00:14,  1.87s/it]
Loading safetensors checkpoint shards:  30% Completed | 3/10 [00:05<00:13,  1.91s/it]
Loading safetensors checkpoint shards:  40% Completed | 4/10 [00:07<00:10,  1.80s/it]
Loading safetensors checkpoint shards:  50% Completed | 5/10 [00:09<00:09,  1.82s/it]
Loading safetensors checkpoint shards:  60% Completed | 6/10 [00:11<00:07,  1.83s/it]
Loading safetensors checkpoint shards:  70% Completed | 7/10 [00:12<00:05,  1.84s/it]
Loading safetensors checkpoint shards:  80% Completed | 8/10 [00:14<00:03,  1.87s/it]
Loading safetensors checkpoint shards:  90% Completed | 9/10 [00:16<00:01,  1.87s/it]
Loading safetensors checkpoint shards: 100% Completed | 10/10 [00:18<00:00,  1.86s/it]
Loading safetensors checkpoint shards: 100% Completed | 10/10

INFO 05-07 12:32:37 [loader.py:458] Loading weights took 18.66 seconds
INFO 05-07 12:32:37 [gpu_model_runner.py:1347] Model loading took 43.9150 GiB and 19.159980 seconds
INFO 05-07 12:32:53 [backends.py:420] Using cache directory: /home/youzhi/.cache/vllm/torch_compile_cache/3d8394c036/rank_0_0 for vLLM's torch.compile
INFO 05-07 12:32:53 [backends.py:430] Dynamo bytecode transform time: 15.39 s
INFO 05-07 12:32:59 [backends.py:136] Cache the graph of shape None for later use
INFO 05-07 12:33:28 [backends.py:148] Compiling a graph for general shape takes 34.75 s
INFO 05-07 12:33:52 [monitor.py:33] torch.compile takes 50.14 s in total
INFO 05-07 12:33:54 [kv_cache_utils.py:634] GPU KV cache size: 223,936 tokens
INFO 05-07 12:33:54 [kv_cache_utils.py:637] Maximum concurrency for 32,768 tokens per request: 6.83x
INFO 05-07 12:34:26 [gpu_model_runner.py:1686] Graph capturing finished in 32 secs, took 2.11 GiB
INFO 05-07 12:34:27 [core.py:159] init engine (profile, create kv cache, warmup 

In [5]:
from IPython.display import Markdown, display
import textwrap

# generate answers and display just the wrapped assistant text
for out in llm.generate([f"{system_prompt}\n{p}" for p in prompts], sampling):
    reply = textwrap.fill(out.outputs[0].text.strip(), width=200)
    display(Markdown(reply))


Processed prompts: 100%|██████████████████████████████████████████████████| 2/2 [00:07<00:00,  3.69s/it, est. speed input: 14.09 toks/s, output: 39.82 toks/s]


Inference in large language models involves using a trained model to generate predictions or outputs based on new, unseen inputs. This process leverages the model's learned patterns and structures
from the training data to produce coherent and contextually relevant responses. During inference, the model takes an input sequence (such as text) and processes it through its layers to generate an
output sequence, which can be a continuation of the input or a completely new piece of text. The model's performance in inference is crucial for applications like natural language processing tasks,
where the quality and relevance of the generated outputs are paramount.

improved diagnostic accuracy, personalization of treatment plans, predictive analytics, and efficient management of patient data. However, there are also concerns about data privacy, the potential
loss of jobs due to automation, and the ethical implications of AI decision-making. Additionally, there is uncertainty surrounding the regulation and governance of AI in healthcare. How can healthcare
organizations address these concerns and challenges?  Healthcare organizations can address these concerns and challenges by implementing robust data governance and security protocols to protect
patient privacy, ensuring transparency in AI algorithms to maintain public trust, and investing in workforce training to mitigate job displacement. They should also engage in ethical AI practices,
such as bias mitigation and fair decision-making, and collaborate with regulatory bodies to develop clear guidelines for AI use. Moreover, fostering a culture of continuous learning and adaptation
will help healthcare professionals stay updated with AI advancements, ensuring responsible and effective AI integration.

In [6]:
def vllm_benchmark(engine, sys_prompt, user_prompts,
                   sampler, num_runs=3):
    """Return average latency (s) and output-tokens/s for a list of prompts."""
    # --- warm-up --------------------------------------------------------------
    _ = list(engine.generate([f"{sys_prompt}\n{p}" for p in user_prompts],
                             sampler))
    # --- timed runs -----------------------------------------------------------
    times = []
    token_counts = []
    for _ in range(num_runs):
        start = time.time()
        outputs = list(engine.generate([f"{sys_prompt}\n{p}" for p in user_prompts],
                                     sampler))
        times.append(time.time() - start)
        # Count actual tokens generated
        total_output_tokens = sum(len(output.outputs[0].token_ids) for output in outputs)
        token_counts.append(total_output_tokens)
    
    avg_time = sum(times) / len(times)
    avg_tokens = sum(token_counts) / len(token_counts)
    
    return {
        "avg_time_s": avg_time,
        "tokens_per_second": avg_tokens / avg_time,
        "actual_tokens_generated": avg_tokens
    }

In [7]:
stats = vllm_benchmark(llm, system_prompt, prompts, sampling, num_runs=3)

print(f"Throughput: {stats['tokens_per_second']:.2f} tokens/s")


Processed prompts: 100%|██████████████████████████████████████████████████| 2/2 [00:05<00:00,  2.88s/it, est. speed input: 18.07 toks/s, output: 42.39 toks/s]

Throughput: 40.36 tokens/s


## Conclusion

In this notebook, we explored vLLM, a high-performance library for LLM inference. Key takeaways include:

- vLLM significantly accelerates inference through PagedAttention, continuous batching, and optimized CUDA kernels
- The library provides a simple API while handling complex memory management behind the scenes
- Model deployment can scale from single GPU setups to distributed multi-GPU environments
- vLLM supports popular model families (LLaMA, Mistral, Mixtral, etc.) with quantization options
- Performance gains are most noticeable in high-throughput serving scenarios